<h1> NOAA weather data </h1>

In [254]:
from __future__ import division
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap
%pylab inline
pylab.rcParams['figure.figsize'] = (12.0, 6.0) # set size of figures"
plt.rcParams.update({'font.size': 24})

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [255]:
good_columns = pd.read_csv('/Users/attiladobi/Documents/NOAA_Et_2014_2017.txt', nrows=0, header=0 , sep=r"\s+").columns

In [332]:
NOAA = pd.read_csv('/Users/attiladobi/Documents/NOAA_Et_2014_2017.txt', skiprows=1, header=None , sep=r"\s+")\
[[0, 1, 2, 3, 5, 7, 9, 11, 13, 15, 16, 17, 18, 19, 20, 21]]
NOAA.columns=good_columns

In [333]:
NOAA.head()

,STN---,WBAN,YEARMODA,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,633300,99999,20140103,60.7,46.4,1027.7,780.5,17.4,5.8,7.8,999.9,69.1*,49.1,0.00I,999.9,0
1,633300,99999,20140104,60.4,40.2,1026.6,779.6,18.0,8.7,15.5,999.9,70.5*,47.3,0.00I,999.9,0
2,633300,99999,20140106,66.5,44.2,1024.7,777.1,14.8,10.2,15.5,999.9,71.6*,48.2,0.00I,999.9,0
3,633300,99999,20140109,64.0,41.8,1026.9,779.8,17.1,8.7,17.5,999.9,71.6*,49.3,0.00I,999.9,0
4,633300,99999,20140111,61.2,46.9,1027.3,780.2,17.1,11.7,15.5,999.9,68.7*,50.7,0.00I,999.9,0


In [334]:
NOAA['STN---'] = NOAA['STN---'].astype(str)
NOAA.rename(columns={'STN---': 'STN'}, inplace=True)
NOAA.MAX = [float(val.strip('*')) for val in NOAA.MAX]
NOAA.MIN = [float(val.strip('*')) for val in NOAA.MIN]
NOAA.PRCP = [float(val.strip('[A,E,I,G]')) for val in NOAA.PRCP]
NOAA = NOAA.replace('99.99', np.nan).replace('999.9', np.nan).replace('9999.9', np.nan)

<h1> Load in station list </h1>

In [335]:
ET_STN= {}

import re
with open('/Users/attiladobi/Documents/NOAA_station_list.txt', 'r') as f:
    for i in range(100):
        f.next()
    for line in f:
        ET_station = line.split('   ET   ')
        if len(ET_station) > 1:
            #print ET_station[1][34:43].strip()
            STN = ET_station[0].split(' ')[0]
            WBAN = ET_station[0].split(' ')[1]
            NAME = ' '.join(ET_station[0].split(' ')[2:]).strip()
            
            CODE = ET_station[1][:9].strip()
            LAT = ET_station[1][9:17].strip()
            LON = ET_station[1][17:26].strip()
            ELEVATION =  ET_station[1][26:34].strip()
            BEGIN =  ET_station[1][34:43].strip()
            END =  ET_station[1][43:].strip()
            
            ET_STN.update({STN: {'WBAN': WBAN, 'NAME': NAME, 'STN': STN, 'CODE': CODE, 'LAT': LAT, 'LON': LON, \
                                'ELEVATION': ELEVATION, 'BEGIN': BEGIN, 'END': END}})

In [336]:
import json
def save_dict_as_json(filename, mapping):
    '''Save dict as json object'''
    with open(filename, 'w') as outfile:
        json.dump(mapping, outfile, indent=4, sort_keys=True, separators=(',', ':'))

def load_dict_from_json(filepath):
    '''given file path to json mapping return a dict'''
    with open(filepath) as data_file:    
        return json.load(data_file)
    
save_dict_as_json('ET_STATION_LIST.json', ET_STN)

<h1> Add station data to NOAA weather data </h1>

In [337]:
NOAA['ELEVATION'] = [ET_STN['%s' % stn]['ELEVATION'] for stn in NOAA['STN']]
NOAA['STATION_NAME'] = [ET_STN[stn]['NAME'] for stn in NOAA['STN']]
NOAA['LAT'] = [ET_STN['%s' % stn]['LAT'] for stn in NOAA['STN']]
NOAA['LON'] = [ET_STN['%s' % stn]['LON'] for stn in NOAA['STN']]

In [338]:
NOAA.to_csv('/Users/attiladobi/Documents/NOAA_ET_LATLON.csv')

In [339]:
NOAA.groupby('STATION_NAME').count()

,STN,WBAN,YEARMODA,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT,ELEVATION,LAT,LON
STATION_NAME,,,,,,,,,,,,,,,,,,,
ARBA MINCH,647,647,647,647,645,0,0,647,110,110,0,647,647,645,0,647,647,647,647
AWASSA (LAKE),724,724,724,724,723,0,0,724,283,283,0,724,724,700,0,724,724,724,724
BAHIR DAR,693,693,693,693,692,687,684,693,582,582,0,693,693,686,0,693,693,693,693
BOLE INTL,1045,1045,1045,1045,1045,854,853,1030,1041,1040,2,1044,1044,963,0,1045,1045,1045,1045
COMBOLCHA/DESSIE,790,790,790,790,789,784,787,790,466,466,0,790,790,767,0,790,790,790,790
DEBRE MARCOS,661,661,661,661,658,406,658,661,381,381,0,661,661,648,0,661,661,661,661
DIRE DAWA INTL,863,863,863,863,863,861,862,863,841,841,0,863,863,825,0,863,863,863,863
GODE,482,482,482,482,0,478,477,482,402,402,0,482,482,480,0,482,482,482,482
GONDER,318,318,318,318,316,312,311,318,100,100,0,318,318,311,0,318,318,318,318


<h3> To join this data with the malaria table the values have to be grouped by Y-m </h3>

In [340]:
NOAA['DATE'] = [pd.to_datetime(val, format='%Y%m%d') for val in NOAA['YEARMODA']]
NOAA['YEAR_MONTH'] = NOAA['DATE'].dt.strftime('%Y-%m')

In [341]:
NOAA.groupby(['STN', 'YEAR_MONTH' ]).mean()

WBAN      YEARMODA       TEMP       DEWP          SLP  \
STN    YEAR_MONTH                                                             
633300 2014-01     99999.0  2.014012e+07  64.450000  40.472222  1026.583333   
       2014-02     99999.0  2.014022e+07  64.136364  47.395455  1024.740909   
       2014-03     99999.0  2.014032e+07  68.620000  44.935000  1025.763158   
       2014-04     99999.0  2.014042e+07  69.380000  44.491667  1027.004000   
       2014-05     99999.0  2.014051e+07  70.072727  49.333333  1026.600000   
       2014-06     99999.0  2.014062e+07  70.642857  46.776190  1026.309524   
       2014-07     99999.0  2.014072e+07  65.082609  54.695652  1026.581818   
       2014-08     99999.0  2.014082e+07  63.911111  56.800000  1025.800000   
       2014-09     99999.0  2.014091e+07  65.200000  51.672000  1025.929167   
       2014-10     99999.0  2.014102e+07  64.730769  49.534615  1026.592308   
       2014-11     99999.0  2.014111e+07  64.757143  50.828571  1027.742857   
       2014-12     99999.0  2.014123e+07  65.000000  38.600000  1026.500000   
       2015-01     99999.0  2.015012e+07  62.558333  40.283333  1027.600000   
       2015-02     99999.0  2.015022e+07  67.388889  44.088889  1025.833333   
       2015-03     99999.0  2.015031e+07  69.555556  36.266667  1026.833333   
       2015-04     99999.0  2.015041e+07  70.655556  43.000000  1025.777778   
       2015-05     99999.0  2.015052e+07  70.350000  51.600000  1026.800000   
       2015-06     99999.0  2.015061e+07  72.520000  44.705000  1026.020000   
       2015-07     99999.0  2.015072e+07  67.200000  54.670588  1026.011765   
       2015-08     99999.0  2.015082e+07  66.396296  57.114815  1026.322222   
       2015-09     99999.0  2.015092e+07  70.988889  52.633333  1026.844444   
       2015-10     99999.0  2.015101e+07  69.357895  55.910526  1027.789474   
       2015-11     99999.0  2.015112e+07  66.437500  49.337500  1026.912500   
       2015-12     99999.0  2.015122e+07  64.088889  51.644444  1028.111111   
       2016-01     99999.0  2.016012e+07  64.090909  53.418182  1027.500000   
       2016-02     99999.0  2.016022e+07  68.666667  43.888889  1028.144444   
       2016-03     99999.0  2.016032e+07  72.664286  52.357143  1027.083333   
       2016-04     99999.0  2.016042e+07  70.152000  52.508000  1026.256000   
       2016-05     99999.0  2.016052e+07  70.744000  46.968000  1027.860000   
       2016-06     99999.0  2.016061e+07  75.370000  43.480000  1027.445000   
...                    ...           ...        ...        ...          ...   
635330 2014-08     99999.0  2.014082e+07  71.850000  59.944444          NaN   
       2014-09     99999.0  2.014091e+07  71.776923  59.946154          NaN   
       2014-10     99999.0  2.014102e+07  69.726087  61.491304          NaN   
       2014-11     99999.0  2.014112e+07  71.750000  59.675000          NaN   
       2014-12     99999.0  2.014123e+07  77.400000  44.700000          NaN   
       2015-01     99999.0  2.015012e+07  76.485714  47.521429          NaN   
       2015-02     99999.0  2.015022e+07  78.581818  51.636364          NaN   
       2015-03     99999.0  2.015031e+07  75.825000  49.950000          NaN   
       2015-04     99999.0  2.015041e+07  72.166667  62.122222          NaN   
       2015-05     99999.0  2.015053e+07  70.466667  62.200000          NaN   
       2015-06     99999.0  2.015061e+07  70.405000  59.870000          NaN   
       2015-07     99999.0  2.015072e+07  68.535294  58.876471          NaN   
       2015-08     99999.0  2.015082e+07  69.836000  57.232000          NaN   
       2015-09     99999.0  2.015092e+07  72.650000  57.387500          NaN   
       2015-10     99999.0  2.015101e+07  70.553333  62.673333          NaN   
       2015-11     99999.0  2.015111e+07  72.420000  61.730000          NaN   
       2015-12     99999.0  2.015122e+07  77.066667  56.566667          NaN   
       2016-01     99999.0  2.016012e+07  77.000000  53.850000          

In [342]:
NOAA.to_csv('/Users/attiladobi/Documents/NOAA_ET_LATLON.csv')

In [345]:
NOAA.columns

Index([u'STN', u'WBAN', u'YEARMODA', u'TEMP', u'DEWP', u'SLP', u'STP',
       u'VISIB', u'WDSP', u'MXSPD', u'GUST', u'MAX', u'MIN', u'PRCP', u'SNDP',
       u'FRSHTT', u'ELEVATION', u'STATION_NAME', u'LAT', u'LON', u'DATE',
       u'YEAR_MONTH'],
      dtype='object')

In [ ]:
from mpl import Polygon

In [346]:
from shapely.geometry import shape

ImportError: No module named shapely.geometry